In [5]:
import torch

!pip install torch-scatter
!pip install torch-sparse
!pip install torch-cluster
!pip install torch-spline-conv
!pip install torch-geometric

In [6]:
from CliquePoolingLayer import *

### Implementation Testing 

In [ ]:
from torch_geometric.data import Data
from torch_sparse import SparseTensor
from torch import Tensor

def test_clique_pool_dual_edge():

  # A  B  C  D  E  F  G  H  I  J
  # 0. 1. 2. 3. 4. 5. 6. 7. 8. 9

  # A.neighbors = [B]             1
  # B.neighbors = [A, C, D].      0, 2, 3
  # C.neighbors = [B, D, G].      1, 3, 6
  # D.neighbors = [B, C, E, F].   1, 2, 4, 5
  # E.neighbors = [D, F, J].      3, 5, 9
  # F.neighbors = [D, E].         3, 4
  # G.neighbors = [C, H, I, J].   2, 7, 8, 9
  # H.neighbors = [G, I, J].      6, 8, 9
  # I.neighbors = [G, H, J].      6, 7, 9
  # J.neighbors = [E, I, G, H].   4, 8, 6, 7


  edge_index = torch.transpose(torch.tensor([
                            [0, 1],
                            [1, 0],
                            [1, 2],
                            [1, 3],
                            [2, 1],
                            [2, 3],
                            [2, 6],
                            [3, 1],
                            [3, 2],
                            [3, 4],
                            [3, 5],
                            [4, 3],
                            [4, 5],
                            [4, 9],
                            [5, 3],
                            [5, 4],
                            [6, 2],
                            [6, 7],
                            [6, 8],
                            [6, 9],
                            [7, 6],
                            [7, 8],
                            [7, 9],
                            [8, 6],
                            [8, 7],
                            [8, 9],
                            [9, 4],
                            [9, 8],
                            [9, 6],
                            [9, 7]], dtype=torch.long), 0, 1)
  x = torch.rand(10, 40)

  data = Data(x=x, edge_index=edge_index.t().contiguous())
  cliquePooling = CliquePooling('avg')
  dual_x, dual_edges_t, _, dual_batch, _= cliquePooling.forward(x, edge_index)

  assert dual_edges_t.tolist() == [[1, 0, 1, 2, 0], [2, 1, 3, 3, 2]], "Hey, something went wrong!"

In [ ]:
test_clique_pool_dual_edge()

### Benchmark testing


#### Proteins dataset


In [ ]:
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool

#https://github.com/rusty1s/pytorch_geometric/blob/cff934ec28fb70d05d434c74111047078667ace8/examples/proteins_topk_pool.py#L25
dataset = TUDataset(root='/tmp/PROTEINS', name='PROTEINS')
dataset = dataset.shuffle()

n = len(dataset) // 10
b_size = 2
test_dataset = dataset[:n]
train_dataset = dataset[n:]
test_loader = DataLoader(test_dataset, batch_size=b_size)
train_loader = DataLoader(train_dataset, batch_size=b_size)


class Net(torch.nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = GCNConv(dataset.num_node_features, 16)
    self.conv2 = GCNConv(16, 32)
    self.pool  = CliquePooling('avg')
    self.conv3 = GCNConv(32, dataset.num_classes, add_self_loops=False)

  def forward(self, data):
    ## 1. Obtain node embeddings:
    x, edge_index, batch = data.x, data.edge_index, data.batch

    # First GCN
    x = self.conv1(x, edge_index)
    x = F.relu(x)
    x = F.dropout(x, training=self.training)
    # Second GCN
    x = self.conv2(x, edge_index)
    x = F.relu(x)
    x = F.dropout(x, training=self.training)
    x, d_edge_index, _, d_batch, _ = self.pool(x, edge_index, None, batch)
    x = F.relu(x)
    x = F.dropout(x, training=self.training)
    # Third GCN
    x = self.conv3(x, d_edge_index)

    ## 2. Readout layer:
    x = global_mean_pool(x, d_batch)

    x = F.log_softmax(x, dim=-1)

    return x


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.001) # lr=0.01, weight_decay=5e-4

def train(epoch):
  model.train()

  loss_all = 0
  for data in train_loader:
    data = data.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, data.y)
    loss.backward()
    loss_all += data.num_graphs * loss.item()
    optimizer.step()
  return loss_all / len(train_dataset)


def test(loader):
  model.eval()

  correct = 0
  for data in loader:
    data = data.to(device)
    pred = model(data).max(dim=1)[1]
    correct += pred.eq(data.y).sum().item()
  return correct / len(loader.dataset)


for epoch in range(1, 500):
  loss = train(epoch)
  train_acc = test(train_loader)
  test_acc = test(test_loader)
  print('Epoch: {:03d}, Loss: {:.5f}, Train Acc: {:.5f}, Test Acc: {:.5f}'.
        format(epoch, loss, train_acc, test_acc))

Epoch: 001, Loss: 0.68174, Train Acc: 0.60180, Test Acc: 0.54054
Epoch: 002, Loss: 0.66391, Train Acc: 0.60180, Test Acc: 0.54054
Epoch: 003, Loss: 0.65438, Train Acc: 0.60180, Test Acc: 0.54054
Epoch: 004, Loss: 0.64462, Train Acc: 0.60180, Test Acc: 0.54054
Epoch: 005, Loss: 0.63916, Train Acc: 0.60180, Test Acc: 0.54054
Epoch: 006, Loss: 0.63755, Train Acc: 0.60180, Test Acc: 0.54054
Epoch: 007, Loss: 0.63598, Train Acc: 0.60180, Test Acc: 0.54054
Epoch: 008, Loss: 0.63155, Train Acc: 0.60180, Test Acc: 0.54054
Epoch: 009, Loss: 0.63062, Train Acc: 0.60180, Test Acc: 0.54054
Epoch: 010, Loss: 0.63143, Train Acc: 0.60180, Test Acc: 0.54054
Epoch: 011, Loss: 0.63137, Train Acc: 0.60180, Test Acc: 0.54054
Epoch: 012, Loss: 0.62981, Train Acc: 0.60180, Test Acc: 0.54054
Epoch: 013, Loss: 0.62712, Train Acc: 0.60180, Test Acc: 0.54054
Epoch: 014, Loss: 0.62867, Train Acc: 0.60180, Test Acc: 0.54054
Epoch: 015, Loss: 0.62739, Train Acc: 0.64471, Test Acc: 0.61261
Epoch: 016, Loss: 0.62395

### Ablation study w.r.t MUTAG dataset
---
Firstly I apply just 3 convolutional layers without pooling and then after 2nd and 3rd convolution layer, I apply one CliquePooling layer 

In [ ]:
# https://colab.research.google.com/drive/1I8a0DfQ3fI7Njc62__mVXUlcAleUclnb?usp=sharing#scrollTo=CN3sRVuaQ88l

import torch
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='data/TUDataset', name='MUTAG')

print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
print(f'Contains self-loops: {data.contains_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Extracting data/TUDataset/MUTAG/MUTAG.zip
Processing...
Done!

Dataset: MUTAG(188):
Number of graphs: 188
Number of features: 7
Number of classes: 2

Data(edge_attr=[38, 4], edge_index=[2, 38], x=[17, 7], y=[1])
Number of nodes: 17
Number of edges: 38
Average node degree: 2.24
Contains isolated nodes: False
Contains self-loops: False
Is undirected: True


In [ ]:
torch.manual_seed(12345)
dataset = dataset.shuffle()

train_dataset = dataset[:150]
test_dataset = dataset[150:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

from torch_geometric.data import DataLoader

b_size = 64

train_loader = DataLoader(train_dataset, batch_size=b_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=b_size, shuffle=False)

Number of training graphs: 150
Number of test graphs: 38


In [ ]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
  def __init__(self, hidden_channels):
    super(GCN, self).__init__()
    torch.manual_seed(12345)
    self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
    self.conv2 = GCNConv(hidden_channels, hidden_channels)
    self.conv3 = GCNConv(hidden_channels, hidden_channels)
    self.lin = Linear(hidden_channels, dataset.num_classes)

  def forward(self, x, edge_index, batch):
    # 1. Obtain node embeddings 
    x = self.conv1(x, edge_index)
    x = x.relu()
    x = F.dropout(x, training=self.training)
    x = self.conv2(x, edge_index)
    x = x.relu()
    x = F.dropout(x, training=self.training)
    x = self.conv3(x, edge_index)

    # 2. Readout layer
    x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

    # 3. Apply a final classifier
    x = F.dropout(x, p=0.5, training=self.training)
    x = self.lin(x)
    
    return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(7, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay = 5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
  model.train()

  for data in train_loader:  # Iterate in batches over the training dataset.
    out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
    loss = criterion(out, data.y)  # Compute the loss.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    optimizer.zero_grad()  # Clear gradients.

def test(loader):
  model.eval()

  correct = 0
  for data in loader:  # Iterate in batches over the training/test dataset.
    out = model(data.x, data.edge_index, data.batch)  
    pred = out.argmax(dim=1)  # Use the class with highest probability.
    correct += int((pred == data.y).sum())  # Check against ground-truth labels.
  return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
  train()
  train_acc = test(train_loader)
  test_acc = test(test_loader)
  print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.3533, Test Acc: 0.2632
Epoch: 002, Train Acc: 0.6800, Test Acc: 0.8158
Epoch: 003, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 004, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 005, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 006, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 007, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 008, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 009, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 010, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 011, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 012, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 013, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 014, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 015, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 016, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 017, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 018, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 019, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 020, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 021, Train Acc: 0.6467, Test Acc:

#### + adding pooling layer

In [ ]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
  def __init__(self, hidden_channels):
    super(GCN, self).__init__()
    torch.manual_seed(12345)
    self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
    self.conv2 = GCNConv(hidden_channels, hidden_channels)
    self.conv3 = GCNConv(hidden_channels, hidden_channels)
    self.lin = Linear(hidden_channels, dataset.num_classes)
    self.pool  = CliquePooling('avg')

  def forward(self, x, edge_index, batch):
    # 1. Obtain node embeddings 
    x = self.conv1(x, edge_index)
    x = x.relu()
    x = self.conv2(x, edge_index)
    d_x, d_edge_index, _, dual_batch, _ = self.pool(x, edge_index, None, batch)
    x = d_x.relu()
    x = self.conv3(x, edge_index)
    d_x, d_edge_index, _, dual_batch, _ = self.pool(x, edge_index, None, batch)
    # 

    # 2. Readout layer
    x = global_mean_pool(d_x, dual_batch)  # [batch_size, hidden_channels]

    # 3. Apply a final classifier
    x = F.dropout(x, p=0.5, training=self.training)
    x = self.lin(x)
    
    return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(7, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
  (pool): CliquePooling(agg_type=avg)
)


In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, eps=0.0001, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
  model.train()

  for data in train_loader:  # Iterate in batches over the training dataset.
    out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
    loss = criterion(out, data.y)  # Compute the loss.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    optimizer.zero_grad()  # Clear gradients.

def test(loader):
  model.eval()

  correct = 0
  for data in loader:  # Iterate in batches over the training/test dataset.
    out = model(data.x, data.edge_index, data.batch)  
    pred = out.argmax(dim=1)  # Use the class with highest probability.
    correct += int((pred == data.y).sum())  # Check against ground-truth labels.
  return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
  train()
  train_acc = test(train_loader)
  test_acc = test(test_loader)
  print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.3533, Test Acc: 0.2632
Epoch: 002, Train Acc: 0.6467, Test Acc: 0.7632
Epoch: 003, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 004, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 005, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 006, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 007, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 008, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 009, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 010, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 011, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 012, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 013, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 014, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 015, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 016, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 017, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 018, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 019, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 020, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 021, Train Acc: 0.6467, Test Acc: